In [1]:
import os
import pandas as pd
import ast
import re
import time
from datetime import datetime
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# AI
from openai import OpenAI
from dotenv import load_dotenv
from sklearn.cluster import KMeans

# Natural Language Processing
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from fuzzywuzzy import fuzz
# import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')

## ChatGPT API Call

In [ ]:
# chatgpt api
load_dotenv('keys.env')
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

# this function call the openai api, input the prompt and return the response in a string
# string is cleaned to only return the python dictionary
def transform(row, prompt):
    # concat directions and the input text
    prompt = f'{prompt}{row}'
    # print(prompt)
    retry = 0

    # call api
    while retry <= 10:
        print(f"Start API call at {datetime.now()}")
        try:
            # Call API
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                timeout=60
            )
            content = str(response.choices[0].message.content)

            # Handle 'null' and 'None' by replacing them with actual Python None
            content = content.replace("null", "None")

            # Extract JSON-like content between first `{` and last `}`
            start = content.find('{')
            end = content.rfind('}')
            if start != -1 and end != -1:
                content = content[start:end+1]
                print(f"retrieved dictionary at {datetime.now()}")
                return content

            else:
                print(f"No valid JSON-like content found in response at {datetime.now()}")

        except Exception as e:
            print(f"Error during API call or processing: {e}")
        
        retry += 1
        print(f"Retrying {retry}...")
        time.sleep(4)

    print("Failed to retrieve dictionary after maximum retries.")
    return None

## Visualization Helpers

In [ ]:
def format_plot(ax, title='', xlabel='', ylabel='', legend_title='', rotation=60):
    if title:
        ax.set_title(title)
    if xlabel:
        ax.set_xlabel(xlabel)
    if ylabel:    
        ax.set_ylabel(ylabel)
    if legend_title:
        ax.legend_.set_title(legend_title)
    plt.setp(ax.get_xticklabels(), rotation=rotation, ha='right')

## Data Preprocessing Helpers

In [10]:
def safe_eval(row):
    if isinstance(row, str):
        return ast.literal_eval(row)
    return row

# Convert dictionary column into new DataFrame columns
def dict_to_cols(df, dict_column):
    # Safely evaluate the dictionary column
    df[dict_column] = df[dict_column].apply(safe_eval)
    
    # Normalize the dictionary column into separate columns
    normalized_df = pd.json_normalize(df[dict_column])
    
    # Combine the normalized columns with the original DataFrame (optional)
    return normalized_df

# this function check if the text has any non-English characters
def contains_non_english(text):
    # Check if the string contains any non-ASCII characters
    # return pd.notna(s) and not bool(re.fullmatch(r'[\x00-\x7F]*', str(s)))
    if re.search(r'[^\x00-\x7F]+', text):  # Matches any non-ASCII characters
        # Check if the text contains characters outside the English alphabet
        # Specifically look for Korean, Japanese, or any other script
        if re.search(r'[^\x00-\x7F\u0000-\u007F]+', text):  # Matches non-ASCII characters
            return True
    return False

# this function remove non-printable characters and replace non-ASC II characters to ASC II
def replace_non_ascii(s):
    # remove non-printable characters
    s = ''.join(ch for ch in s if ch.isprintable())
    # Replace the en dash (–) with a hyphen (-)
    s = s.replace("–", "-")
    # # Replace the ampersand (&) symbol with the word "and"
    # s = s.replace("&", "and")
    return s

# remove unwanted rows by company names and job titles
def refine_rows(df, drop_rows, keep_rows):
    # drop not legit companies
    df = df[~df['companyName'].isin(drop_rows)]
    # keep legit job titles
    df = df[df['title'].str.contains('|'.join(keep_title_rows), case=False, na=False)]
    # remove duplicates
    df = df.drop_duplicates(subset=['companyName', 'title', 'publishedAt'])
    return df

# condense complex job titles into a few well defined ones
def simplify_title(row, title_list):
    if isinstance(row['title'], str):  # Ensure it's a string
        for title in title_list:
            if title.lower() in row['title'].lower():
                return title
    return None  # Return None if not a match or if title is not a string

## Salary Analysis Helper

In [9]:
# this function cleans the salary data for further feature engineering and visualization
# zeros are removed
# The dataset has a salary column obtained by web scraping, and some salary from AI generated values
# fill hourly and yearly salary if a salary column exist, otherwise keep the AI generated values
def clean_salary(df, cols):
    # remove zeros
    df = df.replace(0, None)
    # call parse_and_fill_salary to fill the hourly and yearly salary columns
    df = df.apply(lambda row: parse_and_fill_salary(row), axis=1)
    return df


# this function parse the salary column and fill the hourly and yearly salary columns
def parse_and_fill_salary(row):
    # Function to extract the numeric values from salary strings
    def extract_salary_value(salary_str):
        numbers = re.findall(r'\d{1,3}(?:,\d{3})*(?:\.\d+)?', salary_str)
        # Convert the numbers to float and return the list
        return [float(num.replace(',', '')) for num in numbers]

    # Checking if 'salary' is defined and hourly or yearly salary is missing
    # if row['salary'] is not None:
    if isinstance(row['salary'], str):
        salary_value = extract_salary_value(row['salary'])
        
        # Case 1: If salary is hourly and min/max hourly salary is missing
        if 'hr' in row['salary'] or 'hour' in row['salary']:
            row['min_hourly_salary'] = min(salary_value)
            row['max_hourly_salary'] = max(salary_value)
            row['min_yearly_salary'] = None
            row['max_yearly_salary'] = None
        
        # Case 2: If salary is monthly and min/max yearly salary is missing
        elif 'mo' in row['salary'] or 'month' in row['salary']:
            row['min_yearly_salary'] = min(salary_value)*12
            row['max_yearly_salary'] = max(salary_value)*12
            row['min_hourly_salary'] = None
            row['max_hourly_salary'] = None
        
        # Case 3: If salary is yearly and min/max hourly salary is missing
        elif 'yr' in row['salary'] or 'year' in row['salary']:
            row['min_yearly_salary'] = min(salary_value)
            row['max_yearly_salary'] = max(salary_value)
            row['min_hourly_salary'] = None
            row['max_hourly_salary'] = None
            
    return row

# calculate average salary
def calculate_average_salary(row, col_min, col_max):
    # Check if both specified columns are not NaN
    if pd.notna(row[col_min]) and pd.notna(row[col_max]):
        return (row[col_min] + row[col_max]) / 2
    # If the minimum column is NaN, return the maximum column value
    elif pd.isna(row[col_min]) and pd.notna(row[col_max]):
        return row[col_max]
    # If the maximum column is NaN, return the minimum column value
    elif pd.isna(row[col_max]) and pd.notna(row[col_min]):
        return row[col_min]
    else:
        # If both are NaN, return NaN (or any custom value you'd prefer)
        return None

## Job Skill Analysis Helpers

In [ ]:
# K-means clustering
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenize, remove stopwords, and lemmatize
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    if not words:
        return None

    return ' '.join(words)

def KMeans_words(n, df):
    df = df.copy()
    # df['cleaned_text'] = df.apply(preprocess_text)

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df)

    # Fit KMeans clustering model
    kmeans = KMeans(n_clusters=n, random_state=42)
    df['cluster'] = kmeans.fit_predict(X)

    # grouped_words = df.groupby(['cluster', 'simplified_job_title'])['cleaned_text'].apply(list).reset_index()
    # return grouped_words
    return df['cluster']

def most_frequent_word(text_list):
    # Flatten the list of cleaned text into individual words
    # all_words = " ".join(text_list).split()
    # Count the frequency of each word
    # word_counts = Counter(all_words)
    word_counts = Counter(text_list)
    # Find the most common word
    if word_counts:
        return word_counts.most_common(1)[0][0]
    return None

def get_top_skills(num_clusters, exploded_df, threshold_percentage=50):
    # clean the required skills again for clustering
    exploded_df = exploded_df[exploded_df['cleaned_required_skills'] != 'etl'].copy()
    exploded_df['cleaned_required_skills'] = exploded_df['cleaned_required_skills'].apply(add_qualifier)
    exploded_df = exploded_df[exploded_df['cleaned_required_skills'].notna()].copy()
    
    # us k-means clustring to group similar skill names  
    exploded_df['cluster'] = KMeans_words(n=num_clusters, df=exploded_df['cleaned_required_skills'])
    
    # calculate counts and standard names from the clusters
    grouped_skills = exploded_df.groupby('cluster')['cleaned_required_skills'].apply(list).reset_index()
    grouped_skills['count'] = grouped_skills['cleaned_required_skills'].apply(len)
    grouped_skills['most_freq_skill'] = grouped_skills['cleaned_required_skills'].apply(most_frequent_word)
    
    # Calculate the frequency of the most frequent word in the cluster
    def most_freq_word_count_percentage(skill_list, most_freq_skill):
        return skill_list.count(most_freq_skill) / len(skill_list) * 100
    
    grouped_skills['most_freq_skill_percentage'] = grouped_skills.apply(
        lambda row: most_freq_word_count_percentage(row['cleaned_required_skills'], row['most_freq_skill']),
        axis=1
    )
    
    # Exclude clusters where the most frequent word count is below the threshold percentage
    filtered_skills = grouped_skills[grouped_skills['most_freq_skill_percentage'] >= threshold_percentage]
    
    # get the top 20 skills
    top_skills = filtered_skills.groupby('most_freq_skill')['count'].sum().reset_index()
    top_skills = top_skills.sort_values(by='count', ascending=False).head(20)

    return top_skills

# test and plot 9 different number of clusters
def tune_num_clusters(df):
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    l = len(df['cleaned_required_skills'].unique())
    list_num_clusters = [int(l/10), int(l/9), int(l/8), int(l/7), int(l/6), int(l/5), int(l/4), int(l/3), int(l/2)]
    for ax, k in zip(axes.flat, list_num_clusters):
        grouped_skills=plot_skills(ax, k, df, str(k)+' Clusters')

    plt.tight_layout()
    plt.show()
    # return grouped_skills

# tuning the number of clusters for best result
def plot_skills(ax, num_clusters, exploded_df, title, threshold_percentage=50):
    top_skills = get_top_skills(num_clusters, exploded_df, threshold_percentage)
    # plot them
    sns.barplot(data=top_skills, x='most_freq_skill', y='count', errorbar=None, ax=ax)
    ax.set_title(f'top skills of {title}')
    ax.set_xlabel('skill name')
    plt.setp(ax.get_xticklabels(), rotation=75, ha='right')

def plot_skills_by_title(df):
    title_list = list(df['simplified_job_title'].unique())

    fig, axes = plt.subplots(2, 2, figsize = (16, 16))

    for ax, title in zip(axes.flat, title_list):
        df_title = df[df['simplified_job_title']==title].copy()
        k = int(len(df_title['cleaned_required_skills'].unique())/5)
        plot_skills(ax, k, df_title, title)
    # plt.savefig('skill_by_title.png', dpi=300, bbox_inches='tight')
    plt.tight_layout()
    plt.show()

def add_qualifier(row):
    if row == 'r':
        return 'r language'
    elif row == 'c++':
        return 'c++ language'
    elif ('skill' in row) or ('skills' in row):
        return ' '.join(word for word in row.split() if word not in ['skill', 'skills'])
    else:
        return row
    
# Function to find the best match and group similar names using fuzzy match
def skills_fuzzymatch(skill_list, threshold=80):
    grouped_skills = []  # List to store the groups of similar skills

    for skill in skill_list:
        # Find if the skill is already grouped
        matched = False
        for group in grouped_skills:
            # Compare with the first skill in each group
            if fuzz.ratio(skill, group[0]) >= threshold:
                group.append(skill)
                matched = True
                break
        
        # If no match found, create a new group
        if not matched:
            grouped_skills.append([skill])

    return grouped_skills

def plot_fuzzy_match(df):
    grouped_skills = df.groupby('grouped_skill').size().reset_index(name='count')
    top_skills = grouped_skills.sort_values(by='count', ascending=False).head(20)

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=top_skills, x='grouped_skill', y='count')
    format_plot(ax, title='top skills', xlabel='skill name')
    plt.savefig("skill_fuzzymatch.png", dpi=300, bbox_inches="tight")
    plt.show()
    return top_skills

# assign the grouped skills back to a new column in original DataFrame
def assign_group(skill, grouped_skills):
    for group in grouped_skills:
        if skill in group:
            return most_frequent_word(group)  # Return the first skill in the group as the representative
    return skill  # If no group, return the skill itself

## Location Analysis Helper

In [ ]:
# this function cleans the location names
def clean_location_names(df):
    location_mapping = {
        'New York City Metropolitan Area': 'New York, NY',
        'Albany, New York Metropolitan Area': 'Albany, NY',
        'District of Columbia, United States': 'Washington, DC',
        'San Francisco County, CA': 'San Francisco, CA',
        'San Francisco Bay Area': 'San Francisco, CA',
        'Los Angeles Metropolitan Area': 'Los Angeles, CA',
        'Los Angeles County, CA': 'Los Angeles, CA',
        'Dallas-Fort Worth Metroplex': 'Dallas, TX',
        'Austin, Texas Metropolitan Area': 'Austin, TX',
        'San Antonio, Texas Metropolitan Area': 'San Antonio, TX',
        'Greater Philadelphia': 'Philadelphia, PA',
        'Greater Seattle Area': 'Seattle, WA',
        'Atlanta Metropolitan Area': 'Atlanta, GA',
        'Annapolis Junction, MD': 'Annapolis, MD',
        'Greater Minneapolis-St. Paul Area': 'Minneapolis, MN',
        'Detroit Metropolitan Area': 'Detroit, MI',
        'Charlotte Metro': 'Charlotte, NC',
        'Green Bay, Wisconsin Metropolitan Area': 'Green Bay, WI'
    }
    
    df['cleaned_location'] = df['location'].map(location_mapping).fillna(df['location'])
    
    return df['cleaned_location']